## Header

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import SymLogNorm
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.precision = None  # set as needed
    display.float_format = '{:,.8f}'.format

## Data Import

Importing data to pandas dataframe

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]

df = pd.DataFrame()
for file in files:
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
        
    if "n_moments" in data and ( "n_shift" in data or "n_mag" in data):
        df = pd.concat([df,data])
    
display(df.set_index('key').sort_index(ascending=False))

## Plot

In [ ]:
set_pandas_display_options()
plt.rcParams['figure.figsize'] = [36, 15]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
tfs    = 40 #title font size
lfs    = 40 #label font size
fs     = 40 #font size
cbarfs = 40 #colorbar font size
afs    = 36 #annotation font size
gfs    = 30 #gap font size

linthresh = 10**-5 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale,vmin=None,vmax=10)

phi_labels = [r'$0.1\pi$',r'$0.3\pi$',r'$0.5\pi$',r'$0.7\pi$',r'$0.9\pi$']

In [ ]:
fig = plt.figure()
gs  = fig.add_gridspec(1,3, hspace=0,wspace=0.01)
ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=1))
ax2 = plt.subplot(gs.new_subplotspec((0, 1), colspan=1),sharey=ax1)
ax3 = plt.subplot(gs.new_subplotspec((0, 2), colspan=1),sharey=ax1)

key = '1701265911'

# parameter
tex, size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
theta, mag = df.set_index('key').at[key,'q'], df.set_index('key').at[key,'mag'],
n_shift, min_shift, max_shift = df.set_index('key').at[key,'n_shift'], df.set_index('key').at[key,'min_shift'], df.set_index('key').at[key,'max_shift']
n_energies, n_moments, n_random_states = df.set_index('key').at[key,'n_energies'], df.set_index('key').at[key,'n_moments'], df.set_index('key').at[key,'n_random_states'],
phi, eta = df.set_index('key').at[key,'phi'], df.set_index('key').at[key,'eta']
scale, epsilon = 12, 0.01,

# data
shifts = np.linspace(min_shift, max_shift, n_shift)
emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

dos = []
for i in range(n_shift):
    dos.append( np.load('./data/'+key+'/dos_'+str(i).zfill(4)+'.npy') )
dos = np.array(dos)

Emin = np.amin(emesh)
Emax = np.amax(emesh)

ax1.imshow(dos.T, aspect='auto',norm=norm, extent=(min_shift,max_shift, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True, cmap='Blues');
ax1.set_title(r"(a) $m=0$",fontsize=tfs)
ax1.set_xlabel(r"$\varphi$",fontsize=lfs)
ax1.set_ylabel(r"$E_F$",labelpad=-40,fontsize=lfs)
ax1.set_xticks([0.1*pi,0.3*pi,0.5*pi,0.7*pi,0.9*pi])
ax1.set_xticklabels(phi_labels)
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.set_ylim( (-10.5,7.75))

ax1.annotate(r'$U1$',(0.01,-0.25),color='black',fontsize=gfs)
ax1.annotate(r'$U2$',(0.01,0.35),color='black',fontsize=gfs)
ax1.annotate(r'$U3$',(0.01,1.0),color='black',fontsize=gfs)

ax1.annotate(r'$U1$',(2.85,-0.25),color='black',fontsize=gfs)
ax1.annotate(r'$U2$',(2.85,0.35),color='black',fontsize=gfs)
ax1.annotate(r'$U3$',(2.85,1.0),color='black',fontsize=gfs)

key = '1701273077'

# parameter
tex, size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
theta, shift = df.set_index('key').at[key,'q'], df.set_index('key').at[key,'shift'],
n_mag, min_mag, max_mag = df.set_index('key').at[key,'n_mag'], df.set_index('key').at[key,'min_mag'], df.set_index('key').at[key,'max_mag']
n_energies, n_moments, n_random_states = df.set_index('key').at[key,'n_energies'], df.set_index('key').at[key,'n_moments'], df.set_index('key').at[key,'n_random_states'],
phi, eta = df.set_index('key').at[key,'phi'], df.set_index('key').at[key,'eta']
scale, epsilon = 12, 0.01,

# data
mags = np.linspace(min_mag, max_mag, n_mag)
emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

dos = []
for i in range(n_mag):
    dos.append( np.load('./data/'+key+'/dos_'+str(i).zfill(4)+'.npy') )
dos = np.array(dos)

Emin = np.amin(emesh)
Emax = np.amax(emesh)

ax2.imshow(dos.T, aspect='auto',norm=norm, extent=(min_mag,max_mag, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True, cmap='Blues');
ax2.set_title(r"(b) $\varphi=0$",fontsize=tfs)
ax2.set_xlabel(r"$m$",fontsize=lfs)
ax2.set_ylabel(r"$E_F$",fontsize=lfs)
ax2.set_xticks([0.1,0.3,0.5,0.7,0.9])
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax2.set_ylim( (-11,8))
ax2.get_yaxis().set_visible(False)
ax2.axvline(x = 0.7, color = 'black', linestyle='--',label = r'$0.7$')
#ax2.annotate(r'$0.7$',(0.72,-1.55),color='black',fontsize=afs)
ax2.axvline(x = 1/np.sqrt(3), color = 'dimgray', linestyle='--',label = r'$\frac{1}{\sqrt{3}}$')
ax2.annotate(r'$\frac{1}{\sqrt{3}}$',(0.46,-1.1),color='dimgray',fontsize=afs)
ax2.axvline(x = np.sqrt(3)/2, color = 'dimgray', linestyle='--',label = r'$\frac{\sqrt{3}}{2}$')
ax2.annotate(r'$\frac{\sqrt{3}}{2}$',(0.76,-1.55),color='dimgray',fontsize=afs)

ax2.annotate(r'$U1$',(0.01,-0.25),color='black',fontsize=gfs)
ax2.annotate(r'$U2$',(0.01,0.35),color='black',fontsize=gfs)
ax2.annotate(r'$U3$',(0.01,1.0),color='black',fontsize=gfs)

ax2.annotate(r'$U1^{\prime}$',(0.6,-0.05),color='black',fontsize=gfs)
ax2.annotate(r'$U2^{\prime}$',(0.6,1.1),color='black',fontsize=gfs)

key = '1701346006'

# parameter
tex, size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
theta, mag = df.set_index('key').at[key,'q'], df.set_index('key').at[key,'mag'],
n_shift, min_shift, max_shift = df.set_index('key').at[key,'n_shift'], df.set_index('key').at[key,'min_shift'], df.set_index('key').at[key,'max_shift']
n_energies, n_moments, n_random_states = df.set_index('key').at[key,'n_energies'], df.set_index('key').at[key,'n_moments'], df.set_index('key').at[key,'n_random_states'],
phi, eta = df.set_index('key').at[key,'phi'], df.set_index('key').at[key,'eta']
scale, epsilon = 12, 0.01,

# data
shifts = np.linspace(min_shift, max_shift, n_shift)

emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

dos = []
for i in range(n_shift):
    dos.append( np.load('./data/'+key+'/dos_'+str(i).zfill(4)+'.npy') )
dos = np.array(dos)

Emin = np.amin(emesh)
Emax = np.amax(emesh)

col = ax3.imshow(dos.T, aspect='auto',norm=norm, extent=(min_shift,max_shift, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True, cmap='Blues');
ax3.set_title(r"(c) $m=0.7$",fontsize=tfs)
ax3.set_xlabel(r"$\varphi$",fontsize=lfs)
ax3.set_ylabel(r"$E_F$",fontsize=lfs)
ax3.set_xticks([0.1*pi,0.3*pi,0.5*pi,0.7*pi,0.9*pi])
ax3.set_xticklabels(phi_labels)
ax3.xaxis.set_minor_locator(AutoMinorLocator())
ax3.set_ylim((-11,8))
ax3.get_yaxis().set_visible(False)

ax3.annotate(r'$U1^{\prime}$',(0.01,-0.05),color='black',fontsize=gfs)
ax3.annotate(r'$U2^{\prime}$',(0.01,1.1),color='black',fontsize=gfs)

ax3.annotate(r'$U1$',(2.85,-0.3),color='black',fontsize=gfs)
ax3.annotate(r'$U2$',(2.85,0.4),color='black',fontsize=gfs)
ax3.annotate(r'$U3$',(2.85,1.05),color='black',fontsize=gfs)

for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', length=8, labelsize=fs)
    ax.tick_params(axis='both', which='minor', length=4, labelsize=fs)

cbar = plt.colorbar(col,ax=fig.get_axes(),pad=0.01)
cbar.set_label(label=r'DOS (a.u.)', size=cbarfs)
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fig.tight_layout()
fig.savefig("./plots/DoS_transitions_reg01", dpi=300, bbox_inches = 'tight')
fig.savefig("./plots/lowres/DoS_transitions_reg01", dpi=100, bbox_inches = 'tight')